In [21]:
import torch
import pickle
from torch import nn 
from torch.utils.data import DataLoader
print("PyTorch version:")
print(torch.__version__)
print("GPU Detected:")
# print(torch.cuda.is_available())
print(torch.backends.mps.is_available())

#defining a shortcut function for later:
import os
# using_GPU = os.path.exists('/opt/bin/nvidia-smi')
using_GPU = torch.device("mps")

PyTorch version:
2.2.0
GPU Detected:
True


In [22]:
with open("./data/train_text", "rb") as f:
    train_text = pickle.load(f)
with open("./data/test_text", "rb") as f:
    test_text = pickle.load(f)  

train_text_data = DataLoader(train_text, batch_size=50, shuffle=True)
test_text_data = DataLoader(test_text, batch_size=50, shuffle=True)


In [19]:
class TextClassificationModel(nn.Module):
    def __init__(self, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()